In [ ]:
using Pkg
Pkg.activate(".")
using Plots
using Revise
using NNeutronics
using Interact
using Printf

In [ ]:
println(join(NNeutronics.col_names(), ","))
println(length(NNeutronics.col_names()))

In [ ]:
blanket = Blanket();

In [ ]:
min_flux = 1E-6
min_heating = 1E-6
min_energy = 1E-6

d1 = 0.02
d2 = 1.00
d3 = 0.02
Li6 = 20.0

@manipulate for d1 in blanket.D1, d2 in blanket.D2, d3 in blanket.D3, Li6 in blanket.enrichment
    r1 = d1 + d2 + d3

    # TBR
    x = LinRange(blanket.D1[1], d1, 100)
    x1 = x
    y1 = TBR.(blanket, x, blanket.D2[1], blanket.D3[1], Li6)
    x = LinRange(blanket.D2[1], d2, 100)
    x2 = d1 .+ x
    y2 = TBR.(blanket, d1, x, blanket.D3[1], Li6)
    x = LinRange(blanket.D3[1], d3, 100)
    x3 = d1 .+ d2 .+ x
    y3 = TBR.(blanket, d1, d2, x, Li6)
    final_TBR = TBR(blanket, d1, d2, d3, Li6)
    pt = Plots.plot(vcat(x1, x2, x3), vcat(y1, y2, y3), lw=2, label="", title=@sprintf("TBR %g", final_TBR))
#    hline!(pt, [1.0], label="", color=:green, linestyle=:dash)
#    hline!(pt, [final_TBR], label="", color=:orange, linestyle=:dash)
    vline!(pt, [0.0, d1, d1 + d2, d1 + d2 + d3], label="", color=:red)
    plot!(pt, xlabel="[m]", xlim=[0.0, d1 + d2 + d3], yscale=:log10, ylabel="TBR")

    # escape
    denergy = diff(NNeutronics.energy_edges())
    pe = plot(blanket.energy_grid, leakeage_energy(blanket, d1, d2, d3, Li6, blanket.energy_grid) ./ denergy .+ min_energy, label="", lw=2)
    plot!(pe, blanket.energy_grid, leakeage_energy(blanket, blanket.D1[1], blanket.D2[1], blanket.D3[1], 1.0, blanket.energy_grid) ./ denergy .+ min_energy, color=:black, label="")
    vline!(pe, [14.1], color=:blue, linestyle=:dash, label="")
    plot!(pe, xlabel="MeV", yscale=:log10, xlim=[0, 16], title="Escape energy distribution")

    # heating
    dradial = diff(NNeutronics.radial_edges())
    ph = plot(blanket.radial_grid * r1, heating(blanket, d1, d2, d3, Li6, blanket.radial_grid) .+ min_heating, label="", lw=2)
    vline!(ph, [0.0, d1, d1 + d2, d1 + d2 + d3], label="")
    plot!(ph, xlabel="[m]", xlim=[0, r1], title="Heating", ylabel="eV cm⁻³")#,yscale=:log10)

    # flux
    norm = flux(blanket, d1, d2, d3, Li6, blanket.radial_grid)[1] .+ min_flux
    pf = plot(blanket.radial_grid * r1, (flux(blanket, d1, d2, d3, Li6, blanket.radial_grid) .+ min_flux)/norm, label="", lw=2)
    vline!(pf, [0.0, d1, d1 + d2, d1 + d2 + d3], label="")
    plot!(pf, xlabel="[m]", xlim=[0, r1], title="Normalized flux", ylabel="cm⁻² s⁻¹")#,yscale=:log10)

    plot(pt, pe, ph, pf, layout=4, size=(800, 500))
end

In [ ]:
@show length(blanket.D1)
@show length(blanket.D2)
@show length(blanket.D3)
@show length(blanket.enrichment)
length(blanket.D1) * length(blanket.D2) * length(blanket.D3) * length(blanket.enrichment)

In [ ]:
fieldnames(typeof(blanket))